In [1]:
import numpy as np

In [5]:
from keras.models import Sequential,Model
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Deconvolution2D, Cropping2D
from keras.layers import Input, Add, Dropout, Permute, add

In [13]:
def Convblock(channel_dimension, block_no, no_of_convs) :
    Layers = []
    for i in range(no_of_convs) :
        
        Conv_name = "CONV_"+str(block_no)+"_"+str(i+1)
        
        
        Layers.append(Convolution2D(channel_dimension,kernel_size = (3,3),padding = "same",activation = "relu",name = Conv_name))
    
    Layers.append(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    return Layers

In [15]:
def FCN_16(image_size):
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = (image_size,image_size,3)))
    
    
    for l in Convblock(64,1,2) :
        model.add(l)
    
    for l in Convblock(128,2,2):
        model.add(l)
    
    for l in Convblock(256,3,3):
        model.add(l)
    
    for l in Convblock(512,4,3):
        model.add(l)
    
    for l in Convblock(512,5,3):
        model.add(l)
    
    model.add(Convolution2D(4096,kernel_size=(7,7),padding = "same",activation = "relu",name = "Fully_connected_1"))
    
    model.add(Convolution2D(4096,kernel_size=(1,1),padding = "same",activation = "relu",name = "Fully_connected_2"))
    
    model.add(Convolution2D(21,kernel_size=(1,1),padding="same",activation="relu",name = "Score"))
    return model

output = FCN_16(512)
print output.summary()
    
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_3 (Permute)          (None, 512, 512, 3)       0         
_________________________________________________________________
CONV_1_1 (Conv2D)            (None, 512, 512, 64)      1792      
_________________________________________________________________
CONV_1_2 (Conv2D)            (None, 512, 512, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 256, 256, 64)      0         
_________________________________________________________________
CONV_2_1 (Conv2D)            (None, 256, 256, 128)     73856     
_________________________________________________________________
CONV_2_2 (Conv2D)            (None, 256, 256, 128)     147584    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 128, 128, 128)     0         
__________